In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
import json
import glob
import shutil
import seaborn as sn
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm as tqdm

df = pd.read_csv('../csv_file/PBL_match_result_all_cej_bone_fold2_dropna.csv')

stage_PBLvalue_dict = {'0':15, '1':15, '2':33, '3':33, '2_low':15}
def diff_with_GT(stage, PBLvalue, PBLstage):
    stage = str(stage)[0]
    if PBLvalue - stage_PBLvalue_dict[stage] > 0:
#         0, 1, 2_high case
        return PBLvalue - stage_PBLvalue_dict[stage]
    elif stage == '2':
        return PBLvalue - stage_PBLvalue_dict['2_low']
    elif stage == '3':
        return PBLvalue - stage_PBLvalue_dict['3']
    else:
        print("error")
        print(stage, PBLvalue, PBLstage)

df['diff'] = np.nan
for index, rows in df.iterrows():
    if str(rows['stage'])[0].replace('1', '0') == str(rows['PBL_stage']).replace('1', '0'):
        df.loc[index, 'diff'] = 0
    elif rows['PBL_value'] != 'No_match' and rows['PBL_stage'] != '-999':
        df.loc[index, 'diff'] = diff_with_GT(rows['stage'], float(rows['PBL_value']), rows['PBL_stage'])
    else:
        # value stay np.nan
        pass




In [2]:
def get_ana_img(img_name_list, target_table, SAVE, savepath=None):
    assert len(img_name_list) > 0
    o_image_path_list = glob.glob('../1_o_image/*.PNG')
    s_image_path_list = glob.glob('../1_seg_image/*.PNG')
    seg_fill_image_point_path_list = glob.glob('../2_seg_img_with_cej/*.PNG')

    seg_image_info_path_list = glob.glob('../3_pattern_detection_result/*.PNG')

    assert len(o_image_path_list)*len(s_image_path_list)*len(seg_fill_image_point_path_list)*len(seg_image_info_path_list) > 0, "some path wrong"
    # 路徑錯誤導致檔案為0

    for index, tooth_path in tqdm(enumerate(img_name_list),total=len(img_name_list)):
        image_name = tooth_path.rsplit('_', 1)[0]
        target_o = [img_path for img_path in o_image_path_list if image_name in img_path]
        target_s = [img_path for img_path in s_image_path_list if image_name in img_path]
        target_info = [img_path for img_path in seg_image_info_path_list if image_name in img_path]
        target_pattern = [img_path for img_path in seg_fill_image_point_path_list if image_name in img_path]
        
        target_csv = target_table.loc[(target_table['tooth_path'] == tooth_path)]
        target_csv = target_csv.dropna()
        assert len(target_csv) > 0
        
        target_csv = target_csv[['tooth_side','stage','PBL_predict_tooth','PBL_value','PBL_stage','tooth_path']]
#         print(tabulate(target_csv, headers='keys', tablefmt='psql'))
        
        if target_o != []:
            o_img = cv2.imread(target_o[0],0)
#             plt.figure(figsize=(12,12))
#             plt.subplot(121)
#             plt.imshow(o_img,cmap='gray')
#             plt.subplot(122)
#             plt.imshow(cv2.cvtColor(s_img,cv2.COLOR_BGR2RGB))
#             plt.show()
        for index, row in target_csv.iterrows():
            p_img_name = tooth_path
            
            blank_image = np.zeros((o_img.shape[0], o_img.shape[1], 3), np.uint8)
            target_info = [img_path for img_path in seg_image_info_path_list if p_img_name[:-4] in img_path]
            target_pattern = [img_path for img_path in seg_fill_image_point_path_list if p_img_name[:-4] in img_path]
            target_info_img = blank_image.copy()
            target_pattern_img = blank_image.copy()
            if target_info!=[]:
                target_info_img = cv2.cvtColor(cv2.imread(target_info[0]),cv2.COLOR_BGR2RGB)
            if target_pattern!=[]:
                target_pattern_img = cv2.cvtColor(cv2.imread(target_pattern[0]),cv2.COLOR_BGR2RGB)         
            plt.figure(figsize=(12,21))
            plt.subplot(121)
            plt.imshow(target_info_img)
            diff = diff_with_GT(str(row['stage']), float(row['PBL_value']), row['PBL_stage'])
            
            message = 'Label\t' + str(row['tooth_side']) + '\t' + str(row['stage'])\
                +'\nPred\t' + str(row['PBL_predict_tooth']) + '\t' + str(row['PBL_stage']) \
                + '\n' + str(row['PBL_value'])[:6] + '%\tdiff ' + str(diff)[:6] + '%'
            
            plt.text(20, 130, message.replace('\t', "   "), size=20, weight='bold', color='red') 
            plt.subplot(122)
            plt.imshow(target_pattern_img)
            if SAVE:
                plt.savefig(os.path.join(savepath, p_img_name), dpi=200, bbox_inches='tight')
                plt.close()
            else:
                plt.show()

In [3]:
# len(df[df['stage'] == 1.0])
print(df.columns)
six_wrong_df = df[abs(df['diff']) >= 6]
vc = six_wrong_df['tooth_side'].value_counts()
Molar_up_list = ['1M', '1D', '2M', '2D', '3M', '3D', '14M', '15M', '16M', '14D', '15D', '16D']
In_list = []
for tooth in Molar_up_list:
    if tooth not in vc.index:
        print(tooth)
    else:
        In_list.append(tooth)
print(vc[In_list])
count=0
img_list = []
for index, row in six_wrong_df.iterrows():
    if row['tooth_side'] in Molar_up_list :
        img_list.append(os.path.basename(row['tooth_path']))
        count+=1
print(count, count/len(six_wrong_df))
print(len(six_wrong_df))

Index(['Unnamed: 0', 'Unnamed: 0.1', 'image_name', 'tooth_side', 'stage',
       'PBL_predict_tooth', 'PBL_value', 'PBL_stage', 'tooth_path',
       'CEJ_points', 'Bone_level_points', 'diff'],
      dtype='object')
16M
1M      1
1D      2
2M     12
2D     28
3M      7
3D     34
14M    11
15M    19
14D    30
15D    32
16D     2
Name: tooth_side, dtype: int64
178 0.22194513715710723
802


In [4]:
get_ana_img(img_list, six_wrong_df, True, './save_case')

In [5]:
summary_df = pd.DataFrame(columns=['stage', 'stage_case', 'wrong_case', '6%_case'])

for sta in [0.0, 1.0, 2.0, 3.0]:
    tmp_df = df[df['stage'] == sta]
    assert len(tmp_df) > 0
    count = 0.0
    count6 = 0.0
    for index, rows in tmp_df.iterrows():
        if str(rows['stage'])[0].replace('1', '0') != str(rows['PBL_stage']).replace('1', '0')\
            and rows['PBL_stage'] != 'No_match' and rows['PBL_stage'] != '-999':
            count += 1
        if abs(rows['diff']) >= 6:
            count6 += 1
    summary_df = summary_df.append({'stage': sta, 'wrong_case':count, \
        '6%_case':count6, 'stage_case': len(tmp_df)}, ignore_index=True)

summary_df = summary_df.append(summary_df[0:2].sum().replace({1:'0 and 1'}), ignore_index=True)

print(summary_df)

     stage  stage_case  wrong_case  6%_case
0        0      2004.0       107.0     49.0
1        1      4948.0      1209.0    502.0
2        2      1774.0       456.0    194.0
3        3       699.0        87.0     57.0
4  0 and 1      6952.0      1316.0    551.0


In [6]:
print(summary_df.reindex([4, 2, 3]))

     stage  stage_case  wrong_case  6%_case
4  0 and 1      6952.0      1316.0    551.0
2        2      1774.0       456.0    194.0
3        3       699.0        87.0     57.0


In [7]:
p_summary_df = summary_df.copy()
p_summary_df['6%_/_wrong'] = p_summary_df['6%_case'] / p_summary_df['wrong_case']
p_summary_df['wrong_case'] = p_summary_df['wrong_case'] / p_summary_df['stage_case']
p_summary_df['6%_case'] = p_summary_df['6%_case'] / p_summary_df['stage_case']
print(p_summary_df.reindex([4, 2, 3]))

     stage  stage_case  wrong_case   6%_case  6%_/_wrong
4  0 and 1      6952.0    0.189298  0.079258    0.418693
2        2      1774.0    0.257046  0.109357    0.425439
3        3       699.0    0.124464  0.081545    0.655172


In [8]:
p_summary_df = summary_df.copy()
p_summary_df = p_summary_df.append(summary_df[0:4].sum().replace({6:'sum'}), ignore_index=True)
print(p_summary_df.reindex([4,2,3,5]))
norm_df = p_summary_df / p_summary_df.max()
norm_df['stage'] = p_summary_df['stage']
print()
print(norm_df[['stage', 'stage_case', 'wrong_case', '6%_case']].reindex([4,2,3]))

     stage  stage_case  wrong_case  6%_case
4  0 and 1      6952.0      1316.0    551.0
2        2      1774.0       456.0    194.0
3        3       699.0        87.0     57.0
5      sum      9425.0      1859.0    802.0


TypeError: ufunc 'true_divide' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''